In [ ]:
pip install tensorflow pillow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
import PIL.Image
from pprint import pprint

In [ ]:
image_size = (64, 64)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Datasets/GTSRB/Final_Training/Images_Converted",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Datasets/GTSRB/Final_Training/Images_Converted",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [7]:
data_augmentation = keras.Sequential(
    [
        layers.RandomRotation(0.1),
    ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

In [9]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [10]:
def testImage(file, model_):
    img = PIL.Image.open(file).resize(image_size)

    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model_.predict(img_array, verbose=0)
    
    c = -1
    v = -1
    for i in range(len(predictions[0])):
        if predictions[0][i] > v:
            v = predictions[0][i]
            c = i

    return (c, v)

In [ ]:
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau

def make_model(input_shape, num_classes):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding="same", input_shape=input_shape, kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv2D(32, (3, 3), padding="same", kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())

    model.add(Conv2D(64, (3, 3), padding="same", kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv2D(64, (3, 3), padding="same", kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), padding="same", kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv2D(128, (3, 3), padding="same", kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv2D(128, (3, 3), padding="same", kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Dropout(0.25))

    model.add(Conv2D(256, (3, 3), padding="same", kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv2D(256, (3, 3), padding="same", kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv2D(256, (3, 3), padding="same", kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(512, kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation="softmax"))


    train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        validation_split=0.2
    )
    
    test_datagen = ImageDataGenerator()

    opt = Adam(learning_rate=0.001)
    lr_scheduler = ReduceLROnPlateau(factor=0.1, patience=5)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

model = make_model(input_shape=image_size + (3,), num_classes=43)

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

callbacks = [
    ModelCheckpoint("./save/save_at_{epoch}.h5"),
    EarlyStopping(patience=3),
    ReduceLROnPlateau(factor=0.1, patience=2)
]

opt = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(
    optimizer=opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

epochs = 1
model.fit(
    train_ds, 
    epochs=epochs, 
    callbacks=callbacks, 
    validation_data=val_ds
)

In [ ]:
import os
import tensorflow as tf

test_all_folders = False
folders_to_test = ['1']  # Only relevant if test_all_folders is False

model_path = './save/save_at_24.keras'
print(f"Using model: {model_path}")

model = tf.keras.models.load_model(model_path)
print("Model loaded successfully.")

correct = 0
mean_quality = 0
count = 0
error = []

print("Testing...")

for class_id in os.listdir("Datasets/GTSRB/Final_Test/Sorted_Images"):
    class_folder = os.path.join("Datasets/GTSRB/Final_Test/Sorted_Images", class_id)
    
    if os.path.isdir(class_folder) and (test_all_folders or class_id in folders_to_test):
        print(f"testing folder {class_id}")  # Printing current folder (class)

        folder_correct = 0
        folder_mean_quality = 0
        folder_count = 0
        folder_error = []

        for img_name in os.listdir(class_folder):
            if img_name.endswith('.jpg'):
                img_path = os.path.join(class_folder, img_name)
                
                true_class = int(class_id)

                predicted_class, confidence = testImage(img_path, model)

                folder_count += 1
                folder_mean_quality += confidence

                if predicted_class == true_class:
                    folder_correct += 1
                else:
                    folder_error.append(img_path)
                    print(f"Error: {img_path} predicted class {predicted_class}, actual class {true_class}, confidence: {confidence * 100:.2f}%")

        if folder_count > 0:
            print(f"\nFolder {class_id} results:")
            print(f"Mean quality: {100 * (folder_mean_quality / folder_count):.2f}%, Correct: {folder_correct}/{folder_count} ({100 * folder_correct / folder_count:.2f}%), Wrong: {folder_count - folder_correct}/{folder_count} ({100 * (folder_count - folder_correct) / folder_count:.2f}%)")
        else:
            print(f"No images were processed in folder {class_id}. Please check the directory.")

        correct += folder_correct
        mean_quality += folder_mean_quality
        count += folder_count
        error.extend(folder_error)

if count > 0:
    print(f"\nGlobal results:")
    print(f"Mean quality: {100 * (mean_quality / count):.2f}%, Correct: {correct}/{count} ({100 * correct / count:.2f}%), Wrong: {count - correct}/{count} ({100 * (count - correct) / count:.2f}%)")
else:
    print("No images were processed. Please check the directory.")

if error:
    print("\nError files:")
    print(error)